# I. Prepare data

In [7]:
import os
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import SimpleITK as sitk

**Lấy địa chỉ các file mhd trong subset5**

In [2]:
pathDicom = 'D:\CBD robotics course\data\sub data\subset5'
lstFilesDCM = []
for dirName, subdirList, filelist in os.walk(pathDicom):
    for filename in filelist:
        if '.mhd' in filename.lower():
            lstFilesDCM.append(os.path.join(dirName, filename))

In [3]:
len(lstFilesDCM)

89

In [4]:
lstFilesDCM[0]

'D:\\CBD robotics course\\data\\sub data\\subset5\\1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840553388986847034053.mhd'

In [5]:
candidates = pd.read_csv('D:\CBD robotics course\data\candidates_V2.csv')

**Viết hàm tạo numpy array qua địa chỉ**

In [8]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyimage = sitk.GetArrayFromImage(itkimage)
    return numpyimage

**Tạo dataframe mới bằng các lọc ra seriesuid có trong dataframe candidates**

In [13]:
datasub5 = candidates[0:0]
datasub5

,seriesuid,coordX,coordY,coordZ,class


In [14]:
for i in range(candidates.shape[0]):
    if ('D:\CBD robotics course\data\sub data\subset5\\'+ candidates.seriesuid[i]+'.mhd' in lstFilesDCM):
        datasub5 = datasub5.append(candidates.loc[i])

In [15]:
datasub5.head()

,seriesuid,coordX,coordY,coordZ,class
1068,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,-17.980000,68.200000,-28.73000,0
1069,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,-46.110000,-16.870000,-63.11000,0
1070,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,31.008737,-39.665943,-94.99373,0
1071,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,-61.191895,98.380910,-228.26433,0
1072,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,93.100000,-33.740000,-173.74000,0


In [16]:
datasub5.shape

(75691, 5)

In [17]:
datasub5.to_csv('newdata2.csv')

In [19]:
n= datasub5.groupby('seriesuid').agg({'class': pd.Series.nunique})
len(n[n['class']== 2])

54

**Trong subset này có 54/89 hình có 2 nhãn. Ta đánh những hình có 2 nhãn là nhãn 1**

In [21]:
n= datasub5['seriesuid'].unique()
uid = []
label = []
for i in n:
    uid.append(i)
    if len(datasub5[(datasub5['seriesuid']== i) & (datasub5['class'] == 1)]) >0 and len(datasub5[(datasub5['seriesuid']== i) & (datasub5['class'] == 0)])>0:
        label.append(1)
    else:
        if len(datasub5[(datasub5['seriesuid']== i) & (datasub5['class'] == 0)]) == 0:
            label.append(1)
        else:
            label.append(0)

In [22]:
len(label)

89

In [23]:
print(label)

[0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0]


**Biến 89 hình trong subset5 thành list các array**

In [10]:
Image = [load_itk_image(i) for i in lstFilesDCM]

In [12]:
Image[0].shape

(481, 512, 512)

**Tiến hành tách array từng lớp cắt(slice) trong list trên và đánh nhãn từng lớp cắt**

In [24]:
lst_array_img = []
lst_label = []
for index,img in enumerate(Image):
    for i in img:
        lst_array_img.append(i)
    for i in range(img.shape[0]):
        lst_label.append(label[index])

In [25]:
print('Số lượng lớp cắt của subset5:', len(lst_array_img))
print('Số lượng nhãn của subset5:', len(lst_label))

Số lượng lớp cắt của subset5: 22587
Số lượng nhãn của subset5: 22587


**Resize về kích cỡ 128x128 để tương thích với mô hình đã train trước đó**

In [26]:
X = []
for i in lst_array_img:
    img = cv2.resize(i, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    X.append(img)

In [27]:
X[0].shape

(128, 128)

**One hot encode label**

In [28]:
df = pd.DataFrame({'label': lst_label})
y = pd.get_dummies(data=df, columns=['label'])
y.head()

,label_0,label_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [38]:
df.groupby('label').size()

label
0     7911
1    14676
dtype: int64

Chỉ có 7911 nhãn 0 (35%) trong khi có 14676 nhãn 1 (65%)

**Chuyển X về mảng numpy array**

In [29]:
X = np.array(X)

In [30]:
X.shape

(22587, 128, 128)

Đổi số chiều của X thành 4 chiều

In [34]:
X = np.expand_dims(X, axis=3)
X.shape

(22587, 128, 128, 1)

# II. Evaluate on our trained model

In [32]:
from keras.models import load_model
model = load_model('D:/CBD robotics course/Assignment/Assignment 10/trained_model.h5')

In [35]:
scores = model.evaluate(X, y, batch_size=32, verbose=0)
print('Accuracy: %.2f%%' % (scores[1]*100))

Accuracy: 63.62%


Lập báo cáo phân loại

In [39]:
predited_y = model.predict(X)
y_pred = [np.argmax(i) for i in predited_y]

y_array = y.to_numpy() # vì y đang là 1 dataframe
y_true = [np.argmax(i) for i in y_array]

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.46      0.25      0.32      7911
           1       0.68      0.85      0.75     14676

    accuracy                           0.64     22587
   macro avg       0.57      0.55      0.54     22587
weighted avg       0.60      0.64      0.60     22587

